# 04 Introduction to Fluids Module
---

## Introduction 🌊

### What is Fluids?
[![Fluids](https://img.shields.io/badge/Fluids-8CAAE6?style=flat&logo=Fluids&logoColor=black)](https://pypi.org/project/fluids/) is a free and open-source Python library used for scientific computing and technical computing.

Fluids is open-source software for engineers and technicians working in the fields of chemical, mechanical, or civil engineering. It includes modules for piping, fittings, pumps, tanks, compressible flow, open-channel flow, atmospheric properties, solar properties, particle size distributions, two phase flow, friction factors, control valves, orifice plates and other flow meters, ejectors, relief valves, and more.

The fluids library is designed to be a low-overhead, lightweight repository of engineering knowledge and utilities that relate to fluid dynamics.

Fluids was originally tightly integrated with SciPy and NumPy; today they are optional components used for only a small amount of functionality which do not have pure-Python numerical methods implemented. Fluids targets Python 2.7 and up as well as PyPy2 and PyPy3. Additionally, fluids has been tested by the author to load in IronPython, Jython, and micropython.

While the routines in Fluids are normally quite fast and as efficiently coded as possible, depending on the application there can still be a need for further speed. PyPy provides a substantial speed boost of 6-12 times for most methods. Fluids also supports Numba, a powerful accelerator that works well with NumPy. The Numba interface to fluids also makes it easy to multithread execution as well, avoiding Python GIL issue.

Fluids runs on all operating systems which support Python, is quick to install, and is free of charge. Fluids is designed to be easy to use while still providing powerful functionality. If you need to perform some fluid dynamics calculations, give fluids a try.

In [242]:
import fluids #Importing fluids 
import math #Importing mathematics 
import thermo #Importing thermodynamics for properties

## Problem 1

Water at 30 degrees Celsius flows through a 20 m length of 50 mm plastic (smooth wall) pipe, at a flow rate of 200 L/min.

Calculate the Reynolds number and friction factor.

In [243]:
from fluids.units import  * # * means all fxn are now imported
from thermo.units import Stream # stream
from math import * # * means all fxn are now imported

In [244]:
Q = 0.2*u.m**3/u.min #Defining flow rate
Q

0.2 <Unit('meter ** 3 / minute')>

In [245]:
T = 30*u.degC #Defining temperature
T

30 <Unit('degree_Celsius')>

In [246]:
P = 2*u.bar # assumed pressure in the line
P

2 <Unit('bar')>

In [247]:
water = Stream('water', T=T, P=P, Q=Q) #Defining a variable named water and calling properties from thermo package
water

<Stream, components=['water'], mole fractions=[1.0], mass flow=3.3166445200262986 kg/s, mole flow=184.10174696292805 mol/s, T=303.15 K, P=200000 Pa>

In [248]:
NPS, Di, Do, t = nearest_pipe(Di=0.05*u.m) #Calling data from pipe database
NPS, Di, Do, t

(2 <Unit('dimensionless')>,
 0.05248 <Unit('meter')>,
 0.0603 <Unit('meter')>,
 0.00391 <Unit('meter')>)

In [249]:
v = Q/(pi/4*Di**2) #Defining velocity Volumetirc flow rate / Cross sectional area
v #Velocity is hypothetical

92.45969608160561 <Unit('meter / minute')>

In [250]:
Re = Reynolds(D=Di, rho=water.rho, mu=water.mu, V=v)
print('Reynolds number = %s' %Re)

Reynolds number = 98175.54427341053 dimensionless


In [251]:
fd = friction_factor(Re=Re, eD=_roughness['Glass']/Di)
print('Darcy friction factor = %s' %fd)

Darcy friction factor = 0.018212892385739768 dimensionless


## Problem 2 

Fuel oil at a specific gravity of 0.815 (kinematic viscosity of 2.7 centistokes) flows at 2 inch, schedule 40 steel pipe 100 foot long at a rate of 2 US gallons/second.

Calculate the pressure drop in bars and psi.

In [252]:
#Defining inputs
SG = 0.815 # Specific gravity

In [253]:
rho = SG*999.1*u.kg/u.m**3 # Converting into density
rho

814.2665 <Unit('kilogram / meter ** 3')>

In [254]:
nu = 2.7*u.centistokes # Kinematic viscosity
nu

2.7 <Unit('centistokes')>

In [255]:
mu = nu_mu_converter(rho,  nu=nu) # Converting into abs viscosity
mu

0.0021985195500000007 <Unit('pascal * second')>

In [256]:
Q = 2*u.gal/u.s # Flow rate
Q

2.0 <Unit('gallon / second')>

In [257]:
L = 100*u.foot # Length
L

100 <Unit('foot')>

In [258]:
NPS, D_pipe, Do_pipe, t = nearest_pipe(Di=2*u.inch) #Calling from piping database
NPS, D_pipe, Do_pipe, t

(2 <Unit('dimensionless')>,
 0.05248 <Unit('meter')>,
 0.0603 <Unit('meter')>,
 0.00391 <Unit('meter')>)

In [259]:
v = Q/(pi/4*D_pipe**2) #Defining velocity Q/A
v

924.5969608160561 <Unit('gallon / meter ** 2 / second')>

In [260]:
Re = Reynolds(rho=rho, mu=mu, D=D_pipe, V=v) #Calculating reynolds numbers
Re

68029.24537736106 <Unit('dimensionless')>

In [261]:
fd = friction_factor(Re=Re, eD=0.0018*u.inch/D_pipe) #Calculating friction drop
fd

0.02270215687134065 <Unit('dimensionless')>

In [262]:
K_friction = K_from_f(fd=fd, L=L, D=D_pipe) #Calculating friction
print('Darcy friction factor = %s' %fd)

Darcy friction factor = 0.02270215687134065 dimensionless


In [263]:
dP = dP_from_K(K=K_friction, rho=rho, V=v) #Calculating pressure drop
print('Pressure drop = %s' %dP.to(u.Pa))
print('In imperial, pressure drop = %s' %dP.to(u.psi))

Pressure drop = 65759.12280469094 pascal
In imperial, pressure drop = 9.537554406715385 pound_force_per_square_inch


## Problem 3

Lube oil at 40 C is flowing at the rate of 600 barrels/hr through 60 meters of 8 inch schedule 40 pipe. Find the total line pressure drop in bar.

Specific gravity : 0.90 at 40 C

Viscosity : 450 cP at 40 C

In [264]:
SG = 0.9 #Defining specific gravity
SG

0.9

In [265]:
rho = SG*999.1*u.kg/u.m**3 #Defining rho
rho

899.19 <Unit('kilogram / meter ** 3')>

In [266]:
nu = 450*u.centipoise #Defining visocity
nu

450 <Unit('centipoise')>

In [267]:
Q = 600*u.barrel/u.hr #Defining flow rate
Q

600.0 <Unit('barrel / hour')>

In [268]:
L = 60*u.meter #Defining length
L

60 <Unit('meter')>

In [269]:
NPS, D_pipe, Do_pipe, t = nearest_pipe(Di=8*u.inch) #Calling piping data base
NPS, D_pipe, Do_pipe, t

(10 <Unit('dimensionless')>,
 0.25446 <Unit('meter')>,
 0.273 <Unit('meter')>,
 0.009269999999999999 <Unit('meter')>)

In [270]:
v = Q/(pi/4*D_pipe**2) #Calculating velocity 
v

11798.378467727003 <Unit('barrel / hour / meter ** 2')>

In [271]:
Re = Reynolds(rho=rho, mu=mu, D=D_pipe, V=v) #Calculating reynolds number
Re

40670.93582448257 <Unit('dimensionless')>

In [272]:
fd = friction_factor(Re=Re, eD=0.0018*u.inch/D_pipe) #Calculating friction drop
print('Darcy friction factor = %s' %fd)

Darcy friction factor = 0.022452729812545924 dimensionless


In [273]:
K_friction = K_from_f(fd=fd, L=L, D=D_pipe) #Calculating k friction
dP = dP_from_K(K=K_friction, rho=rho, V=v) #Calculating pressure drop
print('Pressure drop = %s' %dP.to(u.Pa))
print('In metric, pressure drop = %s' %dP.to(u.bar))

Pressure drop = 363.5041410814 pascal
In metric, pressure drop = 0.0036350414108140003 bar


## Problem 3

Carbon dioxide to be conveyed from the top of stripper of ammonia plant to urea plant. Calculate the optimal pipe size required based on the following data.

Flow rate of CO2 = 1000 t/day

Total length of pipe = 800 m

ALlowable pressure drop = 24 kPa

No of 90° elbows = 8

No of butterfly valve = 1

Temperature of gas = 60°C

Viscosity of CO2 gas = 0.016 cP

Pipe schedule = 20

In [274]:
m = 1000*u.US_ton/u.hr
m

1000.0 <Unit('US_ton / hour')>

In [275]:
T = 60*u.degC
T

60 <Unit('degree_Celsius')>

In [276]:
P = 1.2*u.bar
P

1.2 <Unit('bar')>

In [277]:
CO2 = Stream('Carbon-dioxide', T=T,P=P,m=m)
CO2

<Stream, components=['carbon dioxide'], mole fractions=[1.0], mass flow=251.99576111111116 kg/s, mole flow=5725.940106365924 mol/s, T=333.15 K, P=120000 Pa>

In [278]:
rho = CO2.rho
rho

1.9065740510591098 <Unit('kilogram / meter ** 3')>

In [279]:
L = 800*u.m
L

800 <Unit('meter')>

In [280]:
NPS, D_pipe, Do_pipe, t = nearest_pipe(Do=600*u.mm, schedule='20')
D_pipe

0.59094 <Unit('meter')>

In [281]:
Q = m/rho
Q

524.50100191204 <Unit('US_ton * meter ** 3 / hour / kilogram')>

In [282]:
v = Q/(pi/4*D_pipe**2)
v

1912.3600613221886 <Unit('US_ton * meter / hour / kilogram')>

In [283]:
Re = Reynolds(rho=rho, mu=mu, D=D_pipe, V=v)
Re

246961.71613236284 <Unit('dimensionless')>

In [284]:
fd = friction_factor(Re=Re, eD=0.0018*u.inch/D_pipe)
fd

0.015693751698546685 <Unit('dimensionless')>

In [285]:
from fluids import K_butterfly_valve_Crane
K_elbow = bend_rounded(Di=D_pipe, angle=90*u.degrees, fd=fd)
K_butterfly_valve_Crane = K_butterfly_valve_Crane(D=.1,style=0)
K_tot = K_butterfly_valve_Crane + 8*K_elbow
dP = dP_from_K(K=K_tot, rho=rho, V=v)
print('Pressure drop = %s' %dP.to(u.kPa))

Pressure drop = 564.3359369307665 kilopascal


## Problem 5

Calculate optimal line size and pressure drop for instrument air line to supply 849 kg/hr to gas engine at a pressure of 1.2 barg and temperature of 35 °C

Consider line length as 200 m

Consider 10 Nos 90 °C, 2 No Ball Valves, 5 Nos 45° bends, 1 NRV and 1 reducer as fittings

Consider 40 pipe schedule

Consider viscosity of air as 0.01065 cP 

In [304]:
m = 849*u.kg/u.hr
T = 35*u.degC
P = 1.2*u.bar + 1*u.atmosphere

steam = Stream('air', T=T, P=P, m=m)

rho = steam.rho
mu = steam.mu
print('Density = %s' %rho)
print('Viscosity = %s' %mu)

Density = 2.5015640395379566 kilogram / meter ** 3
Viscosity = 1.886526400435946e-05 pascal * second


In [305]:
L = 200*u.m
NPS, D_pipe, Do_pipe, t = nearest_pipe(Do=800*u.mm, schedule='40')
Q = m/rho
v = Q/(pi/4*D_pipe**2)
Re = Reynolds(rho=rho, mu=mu, D=D_pipe, V=v)
fd = friction_factor(Re=Re, eD=0.0018*u.inch/D_pipe)
K_elbow = bend_rounded(Di=D_pipe, angle=90*u.degrees, fd=fd)
K_elbow_45 = bend_rounded(Di=D_pipe, angle=45*u.degrees, fd=fd)
K_globe_valve = K_globe_valve_Crane(D1=0.9*D_pipe, D2=D_pipe, fd=fd)

K_tot = 5*K_globe_valve + 10*K_elbow + K_friction + 5*K_elbow_45
dP = dP_from_K(K=K_tot, rho=rho, V=v)
print('Pressure drop = %s' %dP.to(u.kPa))
print('Optimal pipe size = %s' %Do.to(u.metre))

Pressure drop = 0.004756264944355712 kilopascal
Optimal pipe size = 0.0603 meter
